Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.


# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

- Submission: Sunday, December 17 ([CET](https://www.timeanddate.com/time/zones/cet))
- Reviews: Dies Natalis Solis Invicti ([CET](https://en.wikipedia.org/wiki/Sol_Invictus))

Notes:

- Reviews will be assigned on Monday, December 4
- You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)


In [ ]:
# As a starting point the opponent plays random, or play a winning move if possible (first improvement)
# https://it.wikipedia.org/wiki/Quadrato_magico tic-tac-toe is a problem of picking 3 numbers with sum 15 (makes it cheaper to compute)

In [22]:
# 2 7 6
# 9 5 1
# 4 3 8

MAGIC_SQUARE = [2, 7, 6, 9, 5, 1, 4, 3, 8]
BOARD_SIZE = 3


class TicTacToe:
    def __init__(self) -> None:
        self.board = [2, 0, 1, 0, 2, 1, 0, 0, 1]

    def __str__(self) -> str:
        board_str = ""
        for i, c in enumerate(self.board):
            if i and i % 3 == 0:
                board_str += "\n"
            if c == 0:
                board_str += "-"
            elif c == 1:
                board_str += "X"
            else:
                board_str += "O"
        return board_str